# Transform GeoJSON Properties into Records

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from collections import Counter

import plotly.graph_objs as go
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, json
import urllib.request 

brd_count = {
    'bin': 0,    'bowl': 0, 
    'bucket': 0, 'cup': 0,
    'tire': 0,   'pottedplant': 0, 
    'jar': 0,    'vase': 0
}

## Sameple data in GeoJSON file

In [2]:
with open(os.path.join('..','data','breeding-sites','krabi-detection.geojson'), 'r') as file:
    data = json.load(file)

print(json.dumps(data['features'][0], ensure_ascii=False, indent=4))

{
    "properties": {
        "subdist": "เหนือคลอง",
        "brd_sites": {
            "216": {
                "sum_each": 1,
                "count": {
                    "bucket": 1
                }
            },
            "0": {
                "sum_each": 1,
                "count": {
                    "bucket": 1
                }
            }
        },
        "sum": 2,
        "date": {
            "month": "03",
            "year": "2016"
        },
        "district": "เหนือคลอง",
        "image_name": "7.89442744_99.02596777",
        "province": "กระบี่"
    },
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [
            99.02596777,
            7.89442744
        ]
    }
}


In [3]:
rows = []
for i, feature in enumerate(data['features']):
    
    row = feature['properties'].copy()
    
    lng, lat = feature['geometry']['coordinates']
    row['lng'], row['lat'] = lng, lat
    
    row['date'] = row['date']['year']+'-'+row['date']['month']
    
    for degree in row['brd_sites']:
        
        detected_brd = row['brd_sites'][degree]['count']

        for cls in brd_count:
            if cls not in row:
                row[cls] = 0
            
            if cls in detected_brd:
                row[cls] += detected_brd[cls]
                
    _= row.pop('brd_sites')
#     _= row.pop('directory')
    _= row.pop('image_name')

    rows.append(row)
    
df = pd.DataFrame.from_dict(rows)
df = df.drop('province', axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
df = df.set_index('date')
df = df.sort_index()
df = df['2016']
df.head()
df.tail()
print('Total:',len(df))

,bin,bowl,bucket,cup,district,jar,lat,lng,pottedplant,subdist,sum,tire,vase
date,,,,,,,,,,,,,
2016-01-01,0,0,2,0,เมืองกระบี่,0,8.006397,98.840986,1,อ่าวนาง,3,0,0
2016-01-01,0,0,0,0,เมืองกระบี่,0,8.019386,98.837395,1,อ่าวนาง,1,0,0
2016-01-01,0,0,0,0,เมืองกระบี่,0,8.012092,98.839691,2,อ่าวนาง,2,0,0
2016-01-01,0,0,0,0,เมืองกระบี่,0,8.011447,98.838362,1,อ่าวนาง,1,0,0
2016-01-01,0,0,1,0,เมืองกระบี่,0,8.010250,98.841269,0,อ่าวนาง,1,0,0


,bin,bowl,bucket,cup,district,jar,lat,lng,pottedplant,subdist,sum,tire,vase
date,,,,,,,,,,,,,
2016-12-01,0,0,1,0,ลำทับ,0,8.189570,99.323854,0,ลำทับ,1,0,0
2016-12-01,0,0,1,0,ลำทับ,0,8.186216,99.321156,0,ลำทับ,1,0,0
2016-12-01,0,0,1,0,ลำทับ,0,8.177941,99.314485,0,ลำทับ,1,0,0
2016-12-01,0,0,0,0,ลำทับ,0,8.192055,99.325797,1,ลำทับ,1,0,0
2016-12-01,0,0,1,0,ลำทับ,0,8.168844,99.309304,0,ลำทับ,1,0,0


Total: 16019


## Load subdistrict polygons

In [4]:
with open(os.path.join('..','data','shapefiles','Krabi.geojson'), 'r') as file:
    data_polygon = json.load(file)

In [5]:
data_polygon['features'][0]

{'geometry': {'coordinates': [[[99.06856570381986, 7.214135468572895],
    [99.06804516861584, 7.214268408755317],
    [99.0675736235135, 7.214535334230721],
    [99.06733931895566, 7.215026676938504],
    [99.06685869301705, 7.215146606341588],
    [99.06633047887809, 7.215039262674347],
    [99.06578615826592, 7.215122451023739],
    [99.065252596804, 7.2152322096665],
    [99.06476972026766, 7.215455600246946],
    [99.06433299829561, 7.215785273728864],
    [99.0639172255741, 7.21614321245551],
    [99.06363648947334, 7.216605725802469],
    [99.06343388470987, 7.217109500272441],
    [99.06292017622191, 7.217511550677879],
    [99.06251685650852, 7.217875140128183],
    [99.06209655052167, 7.218220074262132],
    [99.06172748526471, 7.218614753805172],
    [99.0614291939744, 7.219067656828646],
    [99.06124555181435, 7.219579342956525],
    [99.0611312638611, 7.220110242363038],
    [99.0610891604931, 7.220656397116074],
    [99.06129500879602, 7.221089444284946],
    [99.0618071

## Sum number of breeding sites in each subdistrict

In [6]:
df_sum = []
polygons = []
province = 'กระบี่'
for i, feature in enumerate(data_polygon['features']):
    prop = feature['properties']
    province, district, subdist = prop['PV_TN'], prop['AP_TN'], prop['TB_TN']
    province_en, district_en, subdist_en = prop['PV_EN'], prop['AP_EN'], prop['TB_EN']
    value = df[(df.district == district) & (df.subdist == subdist)].copy()
    
    if len(value) == 0:
        continue
    
    total = 0
    tmp_sum = []
    for cls in brd_count:
        tmp_sum.append(value[cls].sum())
        total += value[cls].sum()

    df_sum.append([Counter(list(value.index)).most_common(1)[0][0],
                   province, district, subdist,
                   province_en, district_en, subdist_en,
                  ]+tmp_sum+[total])
    
df_sum = pd.DataFrame.from_records(df_sum)
df_sum.columns = ['date','province','district','subdist',
                  'province_en','district_en','subdist_en',
                  'bin','bowl', 'bucket','cup','jar',
                  'pottedplant','tire','vase','total']
df_sum = df_sum.set_index('date')
df_sum.drop_duplicates(inplace=True)

df_sum.head()
df_sum.tail()
print('Total:',len(df_sum))

df_sum.to_csv(os.path.join('..','data','breeding-sites','Krabi-sum-detection.csv'))

,province,district,subdist,province_en,district_en,subdist_en,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,,,,,,
2016-02-01,กระบี่,อ่าวลึก,นาเหนือ,Krabi,Ao Luek,Na Nuea,307,10,1,9,32,38,1,104,502
2016-02-01,กระบี่,อ่าวลึก,เขาใหญ่,Krabi,Ao Luek,Khao Yai,253,3,0,16,80,55,2,146,555
2016-03-01,กระบี่,ปลายพระยา,คีรีวง,Krabi,Plai Phraya,Khiri Wong,160,5,2,35,87,31,1,175,496
2016-02-01,กระบี่,เขาพนม,หน้าเขา,Krabi,Khao Phanom,Na Khao,263,2,0,21,33,28,1,112,460
2016-02-01,กระบี่,อ่าวลึก,คลองยา,Krabi,Ao Luek,Khlong Ya,214,2,0,18,48,36,1,143,462


,province,district,subdist,province_en,district_en,subdist_en,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,,,,,,
2016-03-01,กระบี่,คลองท่อม,ทรายขาว,Krabi,Khlong Thom,Sai Khao,643,17,2,9,92,77,3,268,1111
2016-04-01,กระบี่,เกาะลันตา,ศาลาด่าน,Krabi,Ko Lanta,Sala Dan,251,10,0,7,15,31,7,172,493
2016-02-01,กระบี่,ปลายพระยา,เขาต่อ,Krabi,Plai Phraya,Khao To,347,6,1,12,10,50,2,151,579
2016-03-01,กระบี่,ปลายพระยา,ปลายพระยา,Krabi,Plai Phraya,Plai Phraya,605,8,4,44,102,73,5,378,1219
2016-02-01,กระบี่,ปลายพระยา,เขาเขน,Krabi,Plai Phraya,Khao Khen,270,1,0,35,38,24,2,139,509


Total: 47
